# Projet de RAG à partir d'un fichier EPUB

In [ ]:
import fitz
import os
#import mistralai
#import langchain_mistralai
from langchain_mistralai.embeddings import MistralAIEmbeddings
from langchain_chroma import Chroma
#from langchain_text_splitters import CharacterTextSplitter
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
import json

conf_file = json.load(open(r"C:\Users\jch_m\Nextcloud2\ConfigPerso\api_key.json"))
MISTRAL_API_KEY = conf_file["API_KEY_Mistal"]
#MISTRAL_API_KEY = os.getenv("MISTRAL_API_KEY")


### RAG sur 1 fichier (IA - Noah Harari)

In [2]:
path_file = r'\\BV-122025\Doc_Partage\Livres\Yuval Noah Harari\21 Lecons pour le XXIeme siecle (55)\21 Lecons pour le XXIeme siecle - Yuval Noah Harari.epub'
text_segment_size = 1000
overlap_size = 200  
persist_db_path = "./db_IA"

doc = fitz.open(path_file)
print("Nombre de pages :", doc.page_count)

#1. REcuperation du contenu du livre
for page_num in range(doc.page_count):
    page = doc.load_page(page_num)
    text = page.get_text()
    print(f"Page {page_num + 1}:\n{text}\n")    

    segments = []
    start = 0
    while start < len(text):
        end = start + text_segment_size
        segment = text[start:end]
        segments.append(segment)
        start += text_segment_size - overlap_size
    print(f"Nombre de segments créés : {len(segments)}")

def get_clean_epub_text(path):
    doc = fitz.open(path)
    text_parts = []
    
    # On définit des mots à bannir pour éviter la bibliographie
    banned_keywords = ["Bloomberg View", "Guardian", "Notes", "Index"]
    
    for page in doc:
        page_text = page.get_text()
        
        # Filtre simple : on ignore les pages qui ressemblent à de la biblio
        if any(word in page_text for word in banned_keywords):
            continue
            
        text_parts.append(page_text)
    
    return "\n".join(text_parts)

# --- Exécution ---
full_text = get_clean_epub_text(path_file)

# Augmentons un peu la taille des morceaux pour donner plus de contexte à l'IA
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500, # Un peu plus grand pour capter le sens
    chunk_overlap=200,
    separators=["\n\n", "\n", ".", "!", "?", " ", ""]
)

# On recrée les documents
docs = [Document(page_content=t) for t in text_splitter.split_text(full_text)]
embeddings = MistralAIEmbeddings(api_key=MISTRAL_API_KEY, model="mistral-embed")

# 3. Création de la base de données vectorielle (en mémoire ici)
vectorstore = Chroma.from_documents(documents=docs, embedding=embeddings,persist_directory=persist_db_path)
print("Base de données prête !")

Nombre de pages : 377
Page 1:


Nombre de segments créés : 0
Page 2:


Nombre de segments créés : 0
Page 3:
© Éditions Albin Michel, 2018
pour la traduction française
 
Édition anglaise parue sous le titre :
21 LESSONS FOR THE 21ST CENTURY
Copyright © Yuval Noah Harari, 2018
Tous droits réservés.
ISBN : 978-2-226-43143-1


Nombre de segments créés : 1
Page 4:
DU MÊME AUTEUR
Sapiens : une brève histoire de l’humanité
Albin Michel, 2015
 
 
Homo deus : une brève histoire du futur
Albin Michel, 2017


Nombre de segments créés : 1
Page 5:
À mon mari Itzik, à ma mère Pnina,
et à ma grand-mère Fanny,
pour leur amour et leur soutien au fil de longues années.


Nombre de segments créés : 1
Page 6:
Introduction
Dans un monde inondé d’informations sans pertinence, le pouvoir
appartient à la clarté. En théorie, chacun peut prendre part au débat
sur l’avenir de l’humanité, mais il n’est pas si facile de garder une
vision claire. Souvent, nous ne nous apercevons même pas qu’un
débat est en cours et

c:\DocPerso\Dev_Python\ProjetsPerso\RAG_livres\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Base de données prête !


#### Question sur IA / Noah Harari

In [3]:

query = "What hope for the humankind future?"
results = vectorstore.similarity_search(query, k=2)

print("\n--- Résultat du test ---")
print(f"il y a {len(results)} résultats pertinents trouvés.\n")
for i, res in enumerate(results):
    print(f"Extrait {i+1}:\n{res.page_content[:200]}...\n")


--- Résultat du test ---
il y a 2 résultats pertinents trouvés.

Extrait 1:
Troisième partie
DÉSESPOIR ET ESPOIR
Confrontée à des défis sans précédent, et à des désaccords
intenses, l’humanité peut cependant être à la hauteur des
circonstances si nous parvenons à dominer nos ...

Extrait 2:
Troisième partie
DÉSESPOIR ET ESPOIR
Confrontée à des défis sans précédent, et à des désaccords
intenses, l’humanité peut cependant être à la hauteur des
circonstances si nous parvenons à dominer nos ...



In [4]:
query = "What prediction for the future"
results = vectorstore.similarity_search(query, k=2)

print("\n--- Résultat du test ---")
print(f"il y a {len(results)} résultats pertinents trouvés.\n")
for i, res in enumerate(results):
    print(f"Extrait {i+1}:\n{res.page_content[:200]}...\n")


--- Résultat du test ---
il y a 2 résultats pertinents trouvés.

Extrait 1:
comportements et la prédiction des décisions et pourront remplacer
les chauffeurs, les banquiers et les avocats.
Dans les toutes dernières décennies, la recherche dans des
domaines tels que les neuros...

Extrait 2:
comportements et la prédiction des décisions et pourront remplacer
les chauffeurs, les banquiers et les avocats.
Dans les toutes dernières décennies, la recherche dans des
domaines tels que les neuros...



## RAG sur plusieurs fichiers (recettes)

In [2]:
files = [
    #r"C:\Users\jch_m\Nextcloud2\Lectures\Joe Beef - Survivre à l’apocalypse, Plus qu’un autre livre de recettes (1001ebooks.club).epub",
    r"C:\Users\jch_m\Nextcloud2\Lectures\Carl Arsenault , Isaac Hub - Tellement Yummy (2024) (1001ebooks.club).epub"
]

text_segment_size = 1000
overlap_size = 150
all_documents = []

# --- 1. Boucle de traitement des fichiers ---
for path in files:
    if not os.path.exists(path):
        print(f"Fichier introuvable : {path}")
        continue
        
    print(f"Traitement de: {os.path.basename(path)}")
    doc = fitz.open(path)
    
    # On extrait tout le texte du livre d'un coup pour un meilleur découpage
    full_book_text = ""
    for page in doc:
        full_book_text += page.get_text()
    
    # --- 2. Découpage intelligent ---
    # On utilise RecursiveCharacterTextSplitter au lieu d'un while manuel
    # car il respecte la structure des phrases et paragraphes.
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=text_segment_size,
        chunk_overlap=overlap_size,
        separators=["\n\n", "\n", ".", " "]
    )

    for page_num in range(len(doc)):
        page = doc[page_num]
        page_text = page.get_text()
        
        if not page_text:
            continue

        # --- EXTRACTION DU TITRE ---
        # On récupère la première ligne de la page comme nom de la recette
        lines = page_text.split('\n')
        recette_nom = lines[0].strip() # La première ligne est le titre

        # On découpe le texte de CETTE page uniquement
        page_chunks = splitter.split_text(page_text)
        
        for chunk in page_chunks:
            all_documents.append(
                Document(
                    page_content=chunk, 
                    metadata={
                        "source": os.path.basename(path), 
                        "page_nb": page_num + 1 , # +1 car l'index commence à 0
                        "recette": recette_nom
                    }
                )
            )
    doc.close()

# --- 3. Vectorisation groupée ---
embeddings = MistralAIEmbeddings(mistral_api_key=MISTRAL_API_KEY, model="mistral-embed")

# On crée la base avec TOUS les documents des deux livres
vectorstore = Chroma.from_documents(
    documents=all_documents,
    embedding=embeddings,
    persist_directory="./db_recettes"
)


Traitement de: Carl Arsenault , Isaac Hub - Tellement Yummy (2024) (1001ebooks.club).epub


c:\DocPerso\Dev_Python\ProjetsPerso\RAG_livres\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


#### Questions sur recettes

In [3]:
# --- 4. Test de recherche ---
query = "Quelle recette yummi à base de riz et carotte?"
results = vectorstore.similarity_search(query, k=3)

print("\n--- Résultats trouvés ---")
for res in results:
    # On affiche la source pour savoir de quel livre vient l'info
    print(f"[{res.metadata['source']}] : page {res.metadata['page_nb']}\n \
Recette: {res.metadata['recette']}\
          \n{res.page_content[:600]}...\n")


--- Résultats trouvés ---
[Carl Arsenault , Isaac Hub - Tellement Yummy (2024) (1001ebooks.club).epub] : page 43
 Recette: CHIPS DE FEUILLES DE RIZ          
devenir molles).
○  Découper les feuilles de riz en morceaux de la taille souhaitée pour les chips.
○  Dans une petite casserole, à feu moyen-vif, chauffer l’huile jusqu’à ce qu’elle
atteigne 180 ˚C (350 ˚F).
○  Frire 2 ou 3 morceaux de feuilles de riz à la fois. Les chips vont gonfler à vue
d’œil.
○  Retirer immédiatement les chips de la casserole et les déposer sur un essuie-tout
qui absorbera le surplus d’huile.
○  Saupoudrer l’assaisonnement sur les chips. Répéter jusqu’à ce que toutes les chips
soient cuites....

[Carl Arsenault , Isaac Hub - Tellement Yummy (2024) (1001ebooks.club).epub] : page 80
 Recette: PÂTES ULTRA RAPIDES AUX FEUILLES DE RIZ          
○  Dans une casserole, à feu moyen-vif, chauffer l’huile 2 minutes. Quand l’huile est
bien chaude, la verser sur le mélange dans le bol de façon à cuire très légèrement
l

In [ ]:
query = "gratin de pomme de terre?"
results = vectorstore.similarity_search(query, k=3)

print("\n--- Résultats trouvés ---")
for res in results:
    # On affiche la source pour savoir de quel livre vient l'info
    print(f"[{res.metadata['source']}] : page {res.metadata['page_nb']}\n \
Recette: {res.metadata['recette']}\
          \n{res.page_content[:600]}...\n")


--- Résultats trouvés ---
[Carl Arsenault , Isaac Hub - Tellement Yummy (2024) (1001ebooks.club).epub] : page 209
 Recette: Édition: Marianne Prairie et Lucie Desaulniers          
numérique) 20240005384| ISBN 9782761963572| ISBN 9782761963589
(livre numérique)
Vedettes-matière: RVM: Cuisine.| RVMGF: Livres de cuisine.
Classification: LCC TX714.A77 2024| CDD 641.5—dc23
09-24
© 2024, Les Éditions de l’Homme,
division du Groupe Sogides inc., filiale de Québecor Média inc.
(Montréal, Québec)
Tous droits réservés
Dépôt légal: 2024
Bibliothèque et Archives nationales du Québec
ISBN (version papier) 978-2-7619-6357-2
ISBN (version numérique) 978-2-7619-6358-9
DISTRIBUTEURS EXCLUSIFS:
Pour le Canada et les États-Unis:
MESSAGERIES ADP inc.*...

[Carl Arsenault , Isaac Hub - Tellement Yummy (2024) (1001ebooks.club).epub] : page 9
 Recette: DESTINATION: VOS CUISINES!          
Notre mission: transmettre notre passion de la cuisine. Chaque page de ce
livre est donc infusée de plaisir, de créativ